In [50]:
import pandas as pd
import numpy as np
import operator
import os
os.chdir('/Users/fabian_coll/Desktop/PyProjects/COVID-Race-Income-Analysis/data/dirty')

In [51]:

csv = pd.read_csv('county_race.csv')
df = pd.DataFrame(csv)

In [52]:
df = df.rename(
    {'Non-Hispanic White': 'White', 
    'Non-Hispanic Black': 'Black',
    'Non-Hispanic American Indian or Alaska Native': 'AmericanIndian/NativeIslander',
    'Non-Hispanic Asian': 'Asian',
    'Non-Hispanic Native Hawaiian or Other Pacific Islander': 'NativeHawaiian/PacificIslander',
    'Total deaths': 'TotalDeaths',
    'Start Date': 'StartDate',
    'End Date': 'EndDate',
    'County Name': 'County',
    'Urban Rural Code': 'RuralCode'}, 
    axis=1)
# Remove unwanted columns
df = df.drop(['StartDate', 'EndDate', 'RuralCode', 'Data as of', 'FIPS State', 'FIPS County', 'Footnote', 'Urban Rural Description'], axis=1)

In [53]:
df['CountyID'] = df.apply(lambda row: str(row['County']+row['State']), axis = 1)
df

,State,County,FIPS Code,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,NativeHawaiian/PacificIslander,Hispanic,Other,CountyID
0,AK,Anchorage Municipality,2020,Distribution of all-cause deaths (%),2624,180,0.580,0.045,0.211,0.057,0.029,0.033,0.046,Anchorage MunicipalityAK
1,AK,Anchorage Municipality,2020,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.150,0.089,NaN,NaN,Anchorage MunicipalityAK
2,AK,Anchorage Municipality,2020,Distribution of population (%),2624,180,0.556,0.052,0.081,0.095,0.026,0.091,0.099,Anchorage MunicipalityAK
3,AL,Baldwin County,1003,Distribution of all-cause deaths (%),2757,234,0.900,0.079,NaN,NaN,NaN,0.013,NaN,Baldwin CountyAL
4,AL,Baldwin County,1003,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,NaN,NaN,NaN,Baldwin CountyAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,WY,Laramie County,56021,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,NaN,0.198,NaN,Laramie CountyWY
2165,WY,Laramie County,56021,Distribution of population (%),1185,116,0.785,0.022,0.006,0.012,0.001,0.149,0.024,Laramie CountyWY
2166,WY,Natrona County,56025,Distribution of all-cause deaths (%),1244,173,0.908,0.008,0.035,NaN,NaN,0.043,NaN,Natrona CountyWY
2167,WY,Natrona County,56025,Distribution of COVID-19 deaths (%),1244,173,0.815,NaN,0.116,NaN,NaN,0.058,NaN,Natrona CountyWY


In [54]:
df_alldeaths = df[(df['Indicator'] == 'Distribution of all-cause deaths (%)')]
df_COVIDdeaths = df[(df['Indicator'] == 'Distribution of COVID-19 deaths (%)')]
df_population = df[(df['Indicator'] == 'Distribution of population (%)')]

In [55]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [56]:
os.chdir('/Users/fabian_coll/Desktop/PyProjects/County_data')
csv = pd.read_csv('cc-est2019-alldata.csv', encoding='latin-1')
countyStats = pd.DataFrame(csv)
countyStats = countyStats[(countyStats['YEAR'] == 12) & (countyStats['AGEGRP'] == 0)]
countyStats['State'] = countyStats.apply(lambda row: us_state_abbrev[row['STNAME']], axis = 1)
countyStats['CountyID'] = countyStats.apply(lambda row: str(row['CTYNAME']+row['State']), axis = 1)
countyStats = countyStats[countyStats['CountyID'].isin(df_COVIDdeaths.CountyID.unique())]


In [57]:
countyStats = countyStats[['CountyID', 'CTYNAME', 'State', 'TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'H_MALE', 'H_FEMALE']]


In [58]:
countyStats['White_Pop'] = countyStats['WA_MALE'] + countyStats['WA_FEMALE']
countyStats['Black_Pop'] = countyStats['BA_MALE'] + countyStats['BA_FEMALE']
countyStats['Asian_Pop'] = countyStats['AA_MALE'] + countyStats['AA_FEMALE']
countyStats['Hispanic_Pop'] = countyStats['H_MALE'] + countyStats['H_FEMALE']
countyStats['AmericanIndian/NativeIslander_Pop'] = countyStats['IA_MALE'] + countyStats['IA_FEMALE']
countyStats['NativeHawaiian/PacificIslander_Pop'] = countyStats['NA_MALE'] + countyStats['NA_FEMALE']
countyStats['Other_Pop'] = countyStats['TOM_MALE'] + countyStats['TOM_FEMALE']
countyStats['Total_Population'] = sum(
    [
    countyStats['White_Pop'], 
    countyStats['Black_Pop'], 
    countyStats['Asian_Pop'], 
    countyStats['Hispanic_Pop'], 
    countyStats['AmericanIndian/NativeIslander_Pop'], 
    countyStats['NativeHawaiian/PacificIslander_Pop'],
    ]
    )

In [59]:
pop_dict = {}
for index, row in countyStats.iterrows():
    pop_dict[row['CountyID']] = [
        row['Total_Population'],
        row['White_Pop'],
        row['Black_Pop'],
        row['Asian_Pop'],
        row['Hispanic_Pop'],
        row['AmericanIndian/NativeIslander_Pop'],
        row['NativeHawaiian/PacificIslander_Pop'],
        row['Other_Pop']
    ]

def get_population(row, val):
    try:
        return int(pop_dict[row['CountyID']][val])
    except KeyError:
        pass

df_COVIDdeaths['Total_Population'] = df_COVIDdeaths.apply(lambda row: get_population(row, 0), axis = 1)
df_COVIDdeaths['White_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 1), axis = 1)
df_COVIDdeaths['Black_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 2), axis = 1)
df_COVIDdeaths['Asian_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 3), axis = 1)
df_COVIDdeaths['Hispanic_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 4), axis = 1)
df_COVIDdeaths['AmericanIndian/NativeIslander_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 5), axis = 1)
df_COVIDdeaths['NativeHawaiian/PacificIslander_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 6), axis = 1)
df_COVIDdeaths['Other_Pop'] = df_COVIDdeaths.apply(lambda row: get_population(row, 7), axis = 1)

ipykernel_launcher:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:23: SettingWithCopyWarning: 
A value is trying t

In [60]:
df_COVIDdeaths

,State,County,FIPS Code,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,...,Other,CountyID,Total_Population,White_Pop,Black_Pop,Asian_Pop,Hispanic_Pop,AmericanIndian/NativeIslander_Pop,NativeHawaiian/PacificIslander_Pop,Other_Pop
1,AK,Anchorage Municipality,2020,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.15,...,NaN,Anchorage MunicipalityAK,290826.0,183670.0,17195.0,28751.0,26992.0,26179.0,8039.0,24166.0
4,AL,Baldwin County,1003,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,...,NaN,Baldwin CountyAL,229601.0,195198.0,19593.0,2380.0,10534.0,1742.0,154.0,4167.0
7,AL,Calhoun County,1015,Distribution of COVID-19 deaths (%),1972,267,0.798,0.191,NaN,NaN,...,NaN,Calhoun CountyAL,115905.0,85322.0,24213.0,1024.0,4614.0,605.0,127.0,2314.0
10,AL,Colbert County,1033,Distribution of COVID-19 deaths (%),885,160,0.788,0.162,NaN,NaN,...,NaN,Colbert CountyAL,55984.0,44440.0,8959.0,374.0,1789.0,382.0,40.0,1046.0
13,AL,Cullman County,1043,Distribution of COVID-19 deaths (%),1422,202,0.975,NaN,NaN,NaN,...,NaN,Cullman CountyAL,86310.0,80188.0,1156.0,607.0,3752.0,539.0,68.0,1210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,WV,Monongalia County,54061,Distribution of COVID-19 deaths (%),2222,284,0.975,NaN,NaN,NaN,...,NaN,Monongalia CountyWV,105549.0,95265.0,4103.0,3545.0,2352.0,218.0,66.0,2415.0
2158,WV,Ohio County,54069,Distribution of COVID-19 deaths (%),1312,200,0.965,NaN,NaN,NaN,...,NaN,Ohio CountyWV,41010.0,38517.0,1517.0,335.0,537.0,91.0,13.0,938.0
2161,WV,Wood County,54107,Distribution of COVID-19 deaths (%),1313,158,0.987,NaN,NaN,NaN,...,NaN,Wood CountyWV,83118.0,80332.0,1034.0,498.0,1018.0,198.0,38.0,1418.0
2164,WY,Laramie County,56021,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,...,NaN,Laramie CountyWY,111394.0,91392.0,2510.0,1352.0,14843.0,1141.0,156.0,2949.0


In [61]:
df_population

,State,County,FIPS Code,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,NativeHawaiian/PacificIslander,Hispanic,Other,CountyID
2,AK,Anchorage Municipality,2020,Distribution of population (%),2624,180,0.556,0.052,0.081,0.095,0.026,0.091,0.099,Anchorage MunicipalityAK
5,AL,Baldwin County,1003,Distribution of population (%),2757,234,0.832,0.086,0.007,0.011,0.001,0.047,0.017,Baldwin CountyAL
8,AL,Calhoun County,1015,Distribution of population (%),1972,267,0.716,0.210,0.004,0.009,0.001,0.041,0.019,Calhoun CountyAL
11,AL,Colbert County,1033,Distribution of population (%),885,160,0.777,0.160,0.006,0.007,0.001,0.032,0.018,Colbert CountyAL
14,AL,Cullman County,1043,Distribution of population (%),1422,202,0.916,0.012,0.006,0.007,0.001,0.045,0.013,Cullman CountyAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,WV,Monongalia County,54061,Distribution of population (%),2222,284,0.883,0.038,0.002,0.033,0.000,0.022,0.022,Monongalia CountyWV
2159,WV,Ohio County,54069,Distribution of population (%),1312,200,0.919,0.035,0.002,0.008,0.000,0.013,0.022,Ohio CountyWV
2162,WV,Wood County,54107,Distribution of population (%),1313,158,0.951,0.012,0.002,0.006,0.000,0.012,0.017,Wood CountyWV
2165,WY,Laramie County,56021,Distribution of population (%),1185,116,0.785,0.022,0.006,0.012,0.001,0.149,0.024,Laramie CountyWY


In [62]:
def get_deathsPer100k(row, raceRow, racePopRow):
    return ((row[raceRow]*row['COVID-19 Deaths']) / row[racePopRow])*100000

def get_totalDeaths_per100k(row):
    return (row['COVID-19 Deaths'] / row['Total_Population'])*100000

def get_population_proportion(row, raceRow):
    return row[raceRow] / row['Total_Population']

def get_highest_deathsPer100k(row):
    row_dict = {
        'White_deaths_per100k': [], 
        'Black_deaths_per100k': [], 
        'AmericanIndian/NativeIslander_deaths_per100k': [], 
        'Asian_deaths_per100k': [], 
        'NativeHawaiian/PacificIslander_deaths_per100k': [], 
        'Hispanic_deaths_per100k': [], 
        'Other_deaths_per100k': []
        }
    for i in row_dict.keys():
        row_dict[i] = row[i]
    max_name = max(row_dict.items(), key = operator.itemgetter(1))[0]
    return max_name.split('_')[0]

def get_highest_proportion(row):
    row_dict = {
        'White_proportion': [], 
        'Black_proportion': [], 
        'AmericanIndian/NativeIslander_proportion': [], 
        'Asian_proportion': [], 
        'NativeHawaiian/PacificIslander_proportion': [], 
        'Hispanic_proportion': [], 
        'Other_proportion': []
        }
    for i in row_dict.keys():
        row_dict[i] = row[i]
    max_name = max(row_dict.items(), key = operator.itemgetter(1))[0]
    return max_name.split('_')[0]
    
    
df_COVIDdeaths['Total_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_totalDeaths_per100k(row), axis = 1)
df_COVIDdeaths['White_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'White', 'White_Pop'), axis = 1)
df_COVIDdeaths['Black_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Black', 'Black_Pop'), axis = 1)
df_COVIDdeaths['AmericanIndian/NativeIslander_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'AmericanIndian/NativeIslander', 'AmericanIndian/NativeIslander_Pop'), axis = 1)
df_COVIDdeaths['Asian_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Asian', 'Asian_Pop'), axis = 1)
df_COVIDdeaths['NativeHawaiian/PacificIslander_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'NativeHawaiian/PacificIslander', 'NativeHawaiian/PacificIslander_Pop'), axis = 1)
df_COVIDdeaths['Hispanic_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Hispanic', 'Hispanic_Pop'), axis = 1)
df_COVIDdeaths['Other_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_deathsPer100k(row, 'Other', 'Other_Pop'), axis = 1)

df_COVIDdeaths['White_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'White_Pop'), axis = 1)
df_COVIDdeaths['Black_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'Black_Pop'), axis = 1)
df_COVIDdeaths['AmericanIndian/NativeIslander_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'AmericanIndian/NativeIslander_Pop'), axis = 1)
df_COVIDdeaths['Asian_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'Asian_Pop'), axis = 1)
df_COVIDdeaths['NativeHawaiian/PacificIslander_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'NativeHawaiian/PacificIslander_Pop'), axis = 1)
df_COVIDdeaths['Hispanic_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'Hispanic_Pop'), axis = 1)
df_COVIDdeaths['Other_proportion'] = df_COVIDdeaths.apply(lambda row: get_population_proportion(row, 'Other_Pop'), axis = 1)

df_COVIDdeaths['Highest_deaths_per100k'] = df_COVIDdeaths.apply(lambda row: get_highest_deathsPer100k(row), axis = 1)
df_COVIDdeaths['County_majority'] = df_COVIDdeaths.apply(lambda row: get_highest_deathsPer100k(row), axis = 1)

df_COVIDdeaths

ipykernel_launcher:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:44: SettingWithCopyWarning: 
A value is trying t

,State,County,FIPS Code,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,...,Other_deaths_per100k,White_proportion,Black_proportion,AmericanIndian/NativeIslander_proportion,Asian_proportion,NativeHawaiian/PacificIslander_proportion,Hispanic_proportion,Other_proportion,Highest_deaths_per100k,County_majority
1,AK,Anchorage Municipality,2020,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.15,...,NaN,0.631546,0.059125,0.090016,0.098860,0.027642,0.092812,0.083094,AmericanIndian/NativeIslander,AmericanIndian/NativeIslander
4,AL,Baldwin County,1003,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,...,NaN,0.850162,0.085335,0.007587,0.010366,0.000671,0.045880,0.018149,Black,Black
7,AL,Calhoun County,1015,Distribution of COVID-19 deaths (%),1972,267,0.798,0.191,NaN,NaN,...,NaN,0.736137,0.208904,0.005220,0.008835,0.001096,0.039808,0.019965,White,White
10,AL,Colbert County,1033,Distribution of COVID-19 deaths (%),885,160,0.788,0.162,NaN,NaN,...,NaN,0.793798,0.160028,0.006823,0.006680,0.000714,0.031956,0.018684,Black,Black
13,AL,Cullman County,1043,Distribution of COVID-19 deaths (%),1422,202,0.975,NaN,NaN,NaN,...,NaN,0.929070,0.013394,0.006245,0.007033,0.000788,0.043471,0.014019,White,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,WV,Monongalia County,54061,Distribution of COVID-19 deaths (%),2222,284,0.975,NaN,NaN,NaN,...,NaN,0.902567,0.038873,0.002065,0.033586,0.000625,0.022283,0.022880,White,White
2158,WV,Ohio County,54069,Distribution of COVID-19 deaths (%),1312,200,0.965,NaN,NaN,NaN,...,NaN,0.939210,0.036991,0.002219,0.008169,0.000317,0.013094,0.022872,White,White
2161,WV,Wood County,54107,Distribution of COVID-19 deaths (%),1313,158,0.987,NaN,NaN,NaN,...,NaN,0.966481,0.012440,0.002382,0.005991,0.000457,0.012248,0.017060,White,White
2164,WY,Laramie County,56021,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,...,NaN,0.820439,0.022533,0.010243,0.012137,0.001400,0.133248,0.026474,Hispanic,Hispanic


In [63]:
os.chdir('/Users/fabian_coll/Desktop/PyProjects/County_data')
csv = pd.read_csv('county_income.csv', encoding='utf-8')
county_income = pd.DataFrame(csv)

In [64]:
county_income = county_income.dropna()
county_income['fips_txt'] = county_income['fips_txt'].astype('int64')

In [65]:
income_dict = {}
for index, row in county_income.iterrows():
    income_dict[row['fips_txt']] = row['median_income']

def get_median_income(row):
    return income_dict[row['FIPS Code']]

df_COVIDdeaths['median_income'] = df_COVIDdeaths.apply(lambda row: get_median_income(row), axis = 1)

ipykernel_launcher:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [66]:
df_COVIDdeaths

,State,County,FIPS Code,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,...,White_proportion,Black_proportion,AmericanIndian/NativeIslander_proportion,Asian_proportion,NativeHawaiian/PacificIslander_proportion,Hispanic_proportion,Other_proportion,Highest_deaths_per100k,County_majority,median_income
1,AK,Anchorage Municipality,2020,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.15,...,0.631546,0.059125,0.090016,0.098860,0.027642,0.092812,0.083094,AmericanIndian/NativeIslander,AmericanIndian/NativeIslander,82512.0
4,AL,Baldwin County,1003,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,...,0.850162,0.085335,0.007587,0.010366,0.000671,0.045880,0.018149,Black,Black,59871.0
7,AL,Calhoun County,1015,Distribution of COVID-19 deaths (%),1972,267,0.798,0.191,NaN,NaN,...,0.736137,0.208904,0.005220,0.008835,0.001096,0.039808,0.019965,White,White,47747.0
10,AL,Colbert County,1033,Distribution of COVID-19 deaths (%),885,160,0.788,0.162,NaN,NaN,...,0.793798,0.160028,0.006823,0.006680,0.000714,0.031956,0.018684,Black,Black,46683.0
13,AL,Cullman County,1043,Distribution of COVID-19 deaths (%),1422,202,0.975,NaN,NaN,NaN,...,0.929070,0.013394,0.006245,0.007033,0.000788,0.043471,0.014019,White,White,50897.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,WV,Monongalia County,54061,Distribution of COVID-19 deaths (%),2222,284,0.975,NaN,NaN,NaN,...,0.902567,0.038873,0.002065,0.033586,0.000625,0.022283,0.022880,White,White,55652.0
2158,WV,Ohio County,54069,Distribution of COVID-19 deaths (%),1312,200,0.965,NaN,NaN,NaN,...,0.939210,0.036991,0.002219,0.008169,0.000317,0.013094,0.022872,White,White,50390.0
2161,WV,Wood County,54107,Distribution of COVID-19 deaths (%),1313,158,0.987,NaN,NaN,NaN,...,0.966481,0.012440,0.002382,0.005991,0.000457,0.012248,0.017060,White,White,49209.0
2164,WY,Laramie County,56021,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,...,0.820439,0.022533,0.010243,0.012137,0.001400,0.133248,0.026474,Hispanic,Hispanic,69613.0


In [67]:
def get_income_quartile(row):
    if row['median_income'] < np.percentile(df_COVIDdeaths['median_income'], 25):
        return 'Low'
    elif row['median_income'] < np.percentile(df_COVIDdeaths['median_income'], 50) and row['median_income'] >= np.percentile(df_COVIDdeaths['median_income'], 25):
        return 'MidLow'
    elif row['median_income'] < np.percentile(df_COVIDdeaths['median_income'], 75) and row['median_income'] >= np.percentile(df_COVIDdeaths['median_income'], 50):
        return 'MidHigh'
    else:
        return 'High'
    
df_COVIDdeaths['median_income_quartile'] = df_COVIDdeaths.apply(lambda row: get_income_quartile(row), axis = 1)

ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [68]:
df_COVIDdeaths

,State,County,FIPS Code,Indicator,TotalDeaths,COVID-19 Deaths,White,Black,AmericanIndian/NativeIslander,Asian,...,Black_proportion,AmericanIndian/NativeIslander_proportion,Asian_proportion,NativeHawaiian/PacificIslander_proportion,Hispanic_proportion,Other_proportion,Highest_deaths_per100k,County_majority,median_income,median_income_quartile
1,AK,Anchorage Municipality,2020,Distribution of COVID-19 deaths (%),2624,180,0.317,NaN,0.339,0.15,...,0.059125,0.090016,0.098860,0.027642,0.092812,0.083094,AmericanIndian/NativeIslander,AmericanIndian/NativeIslander,82512.0,High
4,AL,Baldwin County,1003,Distribution of COVID-19 deaths (%),2757,234,0.859,0.115,NaN,NaN,...,0.085335,0.007587,0.010366,0.000671,0.045880,0.018149,Black,Black,59871.0,MidHigh
7,AL,Calhoun County,1015,Distribution of COVID-19 deaths (%),1972,267,0.798,0.191,NaN,NaN,...,0.208904,0.005220,0.008835,0.001096,0.039808,0.019965,White,White,47747.0,Low
10,AL,Colbert County,1033,Distribution of COVID-19 deaths (%),885,160,0.788,0.162,NaN,NaN,...,0.160028,0.006823,0.006680,0.000714,0.031956,0.018684,Black,Black,46683.0,Low
13,AL,Cullman County,1043,Distribution of COVID-19 deaths (%),1422,202,0.975,NaN,NaN,NaN,...,0.013394,0.006245,0.007033,0.000788,0.043471,0.014019,White,White,50897.0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,WV,Monongalia County,54061,Distribution of COVID-19 deaths (%),2222,284,0.975,NaN,NaN,NaN,...,0.038873,0.002065,0.033586,0.000625,0.022283,0.022880,White,White,55652.0,MidLow
2158,WV,Ohio County,54069,Distribution of COVID-19 deaths (%),1312,200,0.965,NaN,NaN,NaN,...,0.036991,0.002219,0.008169,0.000317,0.013094,0.022872,White,White,50390.0,Low
2161,WV,Wood County,54107,Distribution of COVID-19 deaths (%),1313,158,0.987,NaN,NaN,NaN,...,0.012440,0.002382,0.005991,0.000457,0.012248,0.017060,White,White,49209.0,Low
2164,WY,Laramie County,56021,Distribution of COVID-19 deaths (%),1185,116,0.750,NaN,NaN,NaN,...,0.022533,0.010243,0.012137,0.001400,0.133248,0.026474,Hispanic,Hispanic,69613.0,MidHigh


In [69]:
os.chdir('/Users/fabian_coll/Desktop/PyProjects/COVID-Race-Income-Analysis/data/clean')
df_COVIDdeaths.to_csv('COVID_deaths.csv', index = False)